# Tutorial 25: ReWOO Pattern

In this tutorial, you'll learn how to build token-efficient agents using the ReWOO (Reasoning WithOut Observation) pattern.

**What you'll learn:**
- **Decoupled planning**: Generate complete plan before execution
- **Variable substitution**: Use evidence variables (#E1, #E2) across steps
- **Token efficiency**: Reduce LLM calls from ~10 (ReAct) to 2 (ReWOO)
- **Sequential execution**: Execute tools one by one, building evidence map

By the end, you'll have an agent that plans all steps upfront, executes tools efficiently, and synthesizes a final answer.

## What is ReWOO?

ReWOO is a pattern that **decouples reasoning from observations** to achieve token efficiency:

1. **Planner**: Generate complete plan with evidence variables (1 LLM call)
2. **Executor**: Execute tools sequentially, substituting variables (0 LLM calls for tools)
3. **Solver**: Synthesize final answer from all evidence (1 LLM call)

**Total: 2 LLM calls** vs. ReAct's ~10 interleaved calls.

### Comparison: ReAct vs ReWOO

| Pattern | LLM Calls | Token Usage | Approach |
|---------|-----------|-------------|----------|
| **ReAct** | ~10 | High | Think → Act → Observe → Think → ... |
| **ReWOO** | 2 | Low | Plan All → Execute All → Synthesize |

### Use Cases

- **Multi-step research**: Search, analyze, summarize
- **Data gathering**: Collect information from multiple sources
- **Token-constrained scenarios**: When minimizing LLM calls is critical
- **Predictable workflows**: When the task structure is well-defined

## Architecture

```
┌────────────────────────────────────────────────────────┐
│                   ReWOO Pattern                         │
│                                                         │
│  ┌─────────┐      ┌──────────┐      ┌─────────┐      │
│  │ Planner │─────►│ Executor │─────►│ Solver  │      │
│  │(1 call) │      │  (Loop)  │      │(1 call) │      │
│  └─────────┘      └────┬─────┘      └─────────┘      │
│                        │                               │
│                        └──────┐                        │
│                               ▼                        │
│                         Until All                      │
│                         Steps Done                     │
│                                                         │
│  Plan Example:                                         │
│  Plan: Search for information                          │
│  #E1 = Google[NBA 2024 championship]                   │
│  Plan: Analyze the results                             │
│  #E2 = LLM[Who won according to #E1?]                  │
└────────────────────────────────────────────────────────┘
```

## Setup

In [ ]:
from langgraph_ollama_local import LocalAgentConfig

config = LocalAgentConfig()
print(f"Ollama: {config.ollama.base_url}")
print(f"Model: {config.ollama.model}")

## Step 1: Understand the State

The ReWOO state tracks:
- **task**: Original query
- **plan_string**: Raw plan from planner
- **steps**: Parsed plan as list of (reasoning, var_name, tool, args)
- **results**: Evidence map (e.g., {"#E1": "search results", "#E2": "analysis"})
- **result**: Final answer

In [ ]:
from langgraph_ollama_local.patterns.rewoo import ReWOOState

# Example state
example_state: ReWOOState = {
    "task": "Who won the 2024 NBA championship?",
    "plan_string": "",
    "steps": [],
    "results": {},
    "result": ""
}

print("State defined with:", list(example_state.keys()))

## Step 2: Create the LLM

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,  # Deterministic for planning
)

print("LLM configured")

## Step 3: Create Mock Tools

For this tutorial, we'll create mock tools to demonstrate the pattern without external dependencies.

In [ ]:
from langchain_core.tools import tool

@tool
def mock_search(query: str) -> str:
    """Search engine that returns mock results for demonstration."""
    # Mock responses for demonstration
    if "NBA" in query.upper() or "basketball" in query.lower():
        return """Search Results:
        1. The Boston Celtics won the 2024 NBA Championship, defeating the Dallas Mavericks 4-1 in the Finals.
        2. Jaylen Brown was named the Finals MVP for his outstanding performance.
        3. This was the Celtics' 18th championship, the most in NBA history.
        """
    elif "weather" in query.lower():
        return "Current weather: Sunny, 72°F with light winds."
    else:
        return f"Search results for: {query}\nMock result: Information about {query}"

@tool
def mock_calculator(expression: str) -> str:
    """Simple calculator for demonstration."""
    try:
        # Only allow simple arithmetic for safety
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

# Create tools dictionary
tools = {
    "Google": mock_search,
    "Calculator": mock_calculator,
}

print(f"Tools available: {list(tools.keys())}")

## Step 4: Understand Plan Format

The planner generates plans in this format:

```
Plan: [reasoning for this step]
#E1 = [ToolName][input]

Plan: [reasoning for next step]
#E2 = [ToolName][input that can reference #E1]
```

Let's test the plan parser:

In [ ]:
from langgraph_ollama_local.patterns.rewoo import parse_plan

# Example plan
example_plan = """
Plan: Search for information about the 2024 NBA championship
#E1 = Google[2024 NBA championship winner]

Plan: Analyze the search results to find the Finals MVP
#E2 = LLM[According to #E1, who was the Finals MVP?]
"""

steps = parse_plan(example_plan)

print("Parsed steps:")
for reasoning, var, tool, args in steps:
    print(f"  {var}: {tool}[{args}]")
    print(f"    Reasoning: {reasoning}")
    print()

## Step 5: Create the Graph

Now let's create the complete ReWOO graph:

In [ ]:
from langgraph_ollama_local.patterns.rewoo import create_rewoo_graph, format_tool_descriptions

# Format tool descriptions for the planner
tool_descriptions = format_tool_descriptions(tools)
print("Tool descriptions:")
print(tool_descriptions)
print()

# Create the graph
graph = create_rewoo_graph(llm, tools)

print("ReWOO graph created!")

## Step 6: Visualize the Graph

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not render graph: {e}") 

## Step 7: Run a Simple Task

Let's run a task that requires multiple steps:

In [ ]:
from langgraph_ollama_local.patterns.rewoo import run_rewoo_task

task = "Who won the 2024 NBA championship and who was the Finals MVP?"

print(f"Task: {task}")
print("=" * 60)

result = run_rewoo_task(
    graph,
    task,
    tool_descriptions=tool_descriptions
)

print("\n" + "=" * 60)
print("PLAN:")
print("=" * 60)
print(result["plan_string"])

print("\n" + "=" * 60)
print("EVIDENCE COLLECTED:")
print("=" * 60)
for var, value in result["results"].items():
    print(f"{var}:")
    print(value[:200] + "..." if len(value) > 200 else value)
    print()

print("=" * 60)
print("FINAL ANSWER:")
print("=" * 60)
print(result["result"])

## Step 8: Trace the Execution

Let's step through the execution manually to see how variable substitution works:

In [ ]:
# Show the steps that were parsed from the plan
print("Parsed Steps:")
print("=" * 60)

for i, (reasoning, var, tool, args) in enumerate(result["steps"], 1):
    print(f"Step {i}:")
    print(f"  Reasoning: {reasoning}")
    print(f"  Variable: {var}")
    print(f"  Tool: {tool}")
    print(f"  Input: {args}")
    
    # Show if this step references previous variables
    referenced = [v for v in result["results"].keys() if v in args]
    if referenced:
        print(f"  References: {', '.join(referenced)}")
    
    print()

## Step 9: Test with Different Tasks

Let's try a multi-step calculation task:

In [ ]:
calc_task = "Calculate (15 + 25) * 2 and then add 10 to the result. What's the final answer?"

print(f"Task: {calc_task}")
print("=" * 60)

calc_result = run_rewoo_task(
    graph,
    calc_task,
    tool_descriptions=tool_descriptions
)

print("\n" + "=" * 60)
print("PLAN:")
print("=" * 60)
print(calc_result["plan_string"])

print("\n" + "=" * 60)
print("EVIDENCE:")
print("=" * 60)
for var, value in calc_result["results"].items():
    print(f"{var}: {value}")

print("\n" + "=" * 60)
print("FINAL ANSWER:")
print("=" * 60)
print(calc_result["result"])

## Step 10: Variable Substitution in Action

The key innovation of ReWOO is variable substitution. Let's demonstrate it:

In [ ]:
# Show how variables get substituted
print("Variable Substitution Example:")
print("=" * 60)

if len(result["steps"]) > 1:
    # Get a step that references a previous variable
    for i, (reasoning, var, tool, args) in enumerate(result["steps"]):
        print(f"\nStep {i+1}: {var}")
        print(f"Original input: {args}")
        
        # Show substitution
        substituted = args
        for prev_var, prev_value in result["results"].items():
            if prev_var in args and prev_var != var:
                print(f"\nSubstituting {prev_var}:")
                print(f"  {prev_var} = {prev_value[:100]}..." if len(prev_value) > 100 else f"  {prev_var} = {prev_value}")
                substituted = substituted.replace(prev_var, prev_value)
        
        if substituted != args:
            print(f"\nAfter substitution:")
            print(f"  {substituted[:200]}..." if len(substituted) > 200 else f"  {substituted}")
else:
    print("This task didn't require variable substitution.")

## Step 11: Token Efficiency Analysis

Let's analyze why ReWOO is more token-efficient than ReAct:

In [ ]:
print("Token Efficiency Comparison:")
print("=" * 60)

num_steps = len(result["steps"])

print(f"\nTask: {task}")
print(f"Number of tool executions: {num_steps}")
print()

print("ReWOO Pattern:")
print(f"  1. Planner: 1 LLM call (generates all {num_steps} steps)")
print(f"  2. Executor: {num_steps} tool calls (no LLM calls)")
print(f"  3. Solver: 1 LLM call (synthesizes final answer)")
print(f"  Total LLM calls: 2")
print()

print("ReAct Pattern (for comparison):")
react_calls = num_steps * 2 + 1  # Think before each action + final answer
print(f"  1. Think → Act → Observe (repeated {num_steps} times): {num_steps * 2} LLM calls")
print(f"  2. Final answer: 1 LLM call")
print(f"  Total LLM calls: ~{react_calls}")
print()

efficiency = (1 - 2/react_calls) * 100
print(f"Token savings: ~{efficiency:.0f}% fewer LLM calls with ReWOO!")

## Complete Example: Custom Tools

In [ ]:
# Complete ReWOO implementation with custom tools

from typing_extensions import TypedDict
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langgraph_ollama_local import LocalAgentConfig
from langgraph_ollama_local.patterns.rewoo import (
    create_rewoo_graph,
    run_rewoo_task,
    format_tool_descriptions,
)

# === Custom Tools ===
@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    # Mock implementation
    return f"Weather in {location}: Sunny, 75°F"

@tool  
def get_distance(city1: str, city2: str) -> str:
    """Calculate distance between two cities."""
    # Mock implementation
    return f"Distance from {city1} to {city2}: 250 miles"

# === Setup ===
config = LocalAgentConfig()
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,
)

tools = {
    "Weather": get_weather,
    "Distance": get_distance,
}

# === Create Graph ===
graph = create_rewoo_graph(llm, tools)
tool_desc = format_tool_descriptions(tools)

# === Run Task ===
task = "What's the weather in San Francisco and how far is it from Los Angeles?"

result = run_rewoo_task(graph, task, tool_descriptions=tool_desc)

print("FINAL ANSWER:")
print(result["result"])

## Key Concepts Recap

| Concept | Description |
|---------|-------------|
| **Decoupled Planning** | Generate complete plan before execution |
| **Evidence Variables** | #E1, #E2, etc. store intermediate results |
| **Variable Substitution** | Replace variables with actual values during execution |
| **Token Efficiency** | 2 LLM calls vs. ReAct's ~10 calls |
| **Sequential Execution** | Execute tools one by one, building evidence map |

## When to Use ReWOO

**Use ReWOO when:**
- Token efficiency is critical
- Task structure is predictable
- You can plan all steps upfront
- Tool execution is fast

**Use ReAct when:**
- Need adaptive planning based on results
- Task is highly exploratory
- Tool results are unpredictable

## Variations

1. **Parallel execution**: Execute independent steps in parallel
2. **Replanning**: Add a replanner if initial plan fails
3. **Structured plans**: Use Pydantic models for plan validation
4. **Error handling**: Add retry logic for failed tool calls

## What's Next?

Explore other advanced reasoning patterns:
- **Plan-and-Execute** (Tutorial 21): Adaptive replanning
- **Reflection** (Tutorial 22): Self-improvement loops
- **Reflexion** (Tutorial 23): Learning from failures
- **LATS** (Tutorial 24): Tree search for complex reasoning

## References

- [ReWOO Paper](https://arxiv.org/abs/2305.18323): Original research paper
- [LangGraph ReWOO Tutorial](https://langchain-ai.github.io/langgraph/tutorials/rewoo/rewoo/): Official implementation